#### Notebook to feature engineer fraction of time temperature was above 75th percentile

In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 15)

In [2]:
PATH_1=os.path.join('..','..','..','..','Weather_Data','CAMS','2004','2004_multi_level_26_variables.nc')
ml_26_variables = xr.open_dataset(PATH_1) ## multilevel 26 variables
ml_26_variables = ml_26_variables.squeeze(dim="model_level") ## remove the model_level dimension
ml_26_variables=ml_26_variables.drop_vars('model_level') ## the dropped dimension becomes a column, so remove that
ml_26_variables['longitude'] = ml_26_variables['longitude']-360 ## necessary because of longitude data in multi-level case
temp=ml_26_variables['t'] ## get only the temperature variable
temp

<xarray.DataArray 't' (valid_time: 2928, latitude: 34, longitude: 78)> Size: 31MB
[7765056 values with dtype=float32]
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 23kB 2004-01-01 ... 2004-12-31T21...
  * latitude    (latitude) float64 272B 49.25 48.5 47.75 ... 26.0 25.25 24.5
  * longitude   (longitude) float64 624B -124.8 -124.1 -123.3 ... -67.85 -67.1
Attributes: (12/31)
    GRIB_paramId:                             130
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      2652
    GRIB_typeOfLevel:                         hybrid
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_pv:                                  [0.00000000e+00 2.00000000e+01 ...
    GRIB_shortName:                           t
    GRIB_units:                               K
    long_name:                                Temperature
    units:                                    K
    standard_name:                            air_temperature

In [3]:
temp.to_dataframe()

t
valid_time          latitude longitude                               
2004-01-01 00:00:00 49.25    -124.849999999999994  275.33837890625000
                             -124.099999999999994  274.39990234375000
                             -123.349999999999994  273.89990234375000
                             -122.599999999999994  272.29150390625000
                             -121.849999999999994  269.14208984375000
...                                                               ...
2004-12-31 21:00:00 24.50    -70.100000000000023   295.80035400390625
                             -69.350000000000023   295.71929931640625
                             -68.600000000000023   295.47808837890625
                             -67.850000000000023   295.33746337890625
                             -67.100000000000023   295.21636962890625

[7765056 rows x 1 columns]

In [4]:
PATH_2=os.path.join('..','CAMS_79_variables_2004.pkl')
get_data = pd.read_pickle(PATH_2) ## load the data with all variables
get_data.head()

,NAME,fips,u10,v10,d2m,t2m,bcaod550,duaod550,lsm,msl,omaod550,pm1,pm2p5,pm10,ssaod550,suaod550,z,sp,aod469,aod550,aod670,aod865,aod1240,tcco,tc_c2h6,tchcho,tc_h2o2,tc_oh,tc_c5h8,tc_ch4,tc_hno3,tcno2,tc_no,gtco3,tc_pan,tc_c3h8,tcso2,tcwv,co,aermr04,aermr05,aermr06,c2h6,hcho,h2o2,aermr09,aermr07,aermr10,aermr08,oh,c5h8,hno3,no2,no,go3,pan,c3h8,aermr01,aermr02,aermr03,q,aermr11,so2,t,lai_hv,lai_lv,asn,sd,tcw,aermssdus,aermssdum,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
1536,Autauga,1001,0.00683337287208,0.057042030275763,285.138045866710684,291.443184922078387,0.006202600115014,0.009350168505988,0.986464426073628,101868.908810690394603,0.081677157930176,0.000000014163613,0.000000017921813,0.00000002488742,0.011048199684095,0.102476458644429,1368.669990441114351,100208.716960676189046,0.256863193338604,0.210754607014748,0.161706704817158,0.111922322144872,0.063941478970087,0.000903709647721,0.000003556730764,0.0000036982781,0.000008639873871,0.000000004882142,0.000002406678124,0.009780456917745,0.000005981965344,0.000003607079817,0.000000499659643,0.006222523236384,0.000008102530556,0.000000686161993,0.000004666765427,26.583385756850102,0.00000020613827,0.000000000606878,0.000000001060804,0.000000000319718,0.000000000640074,0.000000003574139,0.000000000787812,0.00000000020121,0.000000009020076,0.00000000023008,0.000000005196208,0.000000000000026,0.000000016054687,0.000000001219707,0.000000008894137,0.000000000805489,0.000000058217727,0.000000002936366,0.000000000301613,0.000000000030549,0.000000002482078,0.000000000406637,0.009771011939628,0.000000003884493,0.000000008757726,291.229120727357952,3.904838410167652,2.618861225907442,0.8798772904946,0.000142324230262,26.733966370773523,0.000001882228055,0.000003818280399,0.000002521341416,0.000000133695621,0.000002882370888,0.000000523901647,0.000013266690496,0.000000264924033,0.000021605929932,0.000001776177146,0.000010768105054,0.000002767577533
441,Baldwin,1003,-0.686377204991297,0.098709684610534,289.161315654784687,293.210229148162625,0.0065176512015,0.011350332024582,0.422456701645339,101820.192886616743635,0.075270206101069,0.000000010275327,0.000000014373656,0.000000020323407,0.017409210229545,0.099257480691872,170.454777918456614,101612.833366256410955,0.254066999867056,0.209805009603284,0.162856597995704,0.115497832684642,0.069419153902784,0.000879848416591,0.000003496830118,0.00000290174518,0.00001000410033,0.000000004910656,0.000000810059373,0.009921953126922,0.000006099906149,0.000002706581563,0.000000402748707,0.006099854150722,0.000006719706327,0.000000627322392,0.000003543048968,29.541384271980043,0.000000170072746,0.000000000701595,0.000000001087802,0.000000000304057,0.000000000560938,0.000000002068646,0.000000000951142,0.000000000173986,0.000000006132297,0.000000000158084,0.000000002853213,0.000000000000042,0.000000004604515,0.000000001429939,0.000000005380378,0.000000000331699,0.000000065733112,0.000000002109738,0.000000000240895,0.000000000110929,0.000000009244196,0.000000006449094,0.012159069345353,0.000000003556322,0.000000004784633,293.368497758160345,2.49388445810583,1.556522709955439,0.879914104610446,0.000002799221276,29.661747316088178,0.000002283485615,0.00000463665722,0.000002998982877,0.000000113949479,0.000002130915398,0.000000576818859,0.000013104885777,0.00000040528174,0.0000332887027,0.000005531420226,0.000011048075714,0.000001624110399
2221,Barbour,1005,0.048569736328535,-0.043343641215073,285.085626002230583,291.984456117338539,0.006371615085981,0.009628423411047,0.986926072992247,101865.836891937535256,0.083274563763094,0.000000013593435,0.00000001728314,0.000000023947229,0.011384691679316,0.100090513804211,1263.685568325176973,100333.562370818894124,0.256831672234675,0.210749819197605,0.16179896305805,0.112183666388715,0.064333678003,0.000901828991918,0.000003539606269,0.000003759553267,0.000008895697339,0.000000004874058,0.000002543291747,0.0097942916769,0.000006009402603,0.

In [5]:
percentile_75 = get_data['t'].quantile(0.75) ## get temperature data
percentile_75
threshold=percentile_75

In [6]:
## use county shapefile from 2008 as the shapefile before 2008 could not be found
SHAPE_PATH=os.path.join('..','..','..','..','Shapefiles','county_shapefiles','2008_county_shapefile','tl_2008_us_county.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,CNTYIDFP,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,geometry
0,19,175,00465276,19175,Union,Union County,06,H1,G4020,None,None,None,A,"POLYGON ((-94.47051 40.97504, -94.47050 40.975..."
1,19,177,00465277,19177,Van Buren,Van Buren County,06,H1,G4020,None,None,None,A,"POLYGON ((-91.96059 40.90070, -91.95925 40.900..."
2,20,097,00485013,20097,Kiowa,Kiowa County,06,H1,G4020,None,None,None,A,"POLYGON ((-99.01471 37.67895, -99.01473 37.678..."
3,20,109,00485019,20109,Logan,Logan County,06,H1,G4020,None,None,None,A,"POLYGON ((-100.89562 39.13329, -100.89404 39.1..."
4,20,003,00484971,20003,Anderson,Anderson County,06,H1,G4020,None,None,None,A,"POLYGON ((-95.51741 38.20619, -95.51741 38.206..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,35,015,00936829,35015,Eddy,Eddy County,06,H1,G4020,None,16100,None,A,"POLYGON ((-104.09554 32.00000, -104.09586 32.0..."
3229,40,015,01101795,40015,Caddo,Caddo County,06,H1,G4020,None,None,None,A,"POLYGON ((-98.36984 35.55133, -98.36964 35.551..."
3230,40,025,01101800,40025,Cimarron,Cimarron County,06,H1,G4020,None,None,None,A,"POLYGON ((-102.14252 36.50032, -102.14380 36.5..."
3231,40,069,01101822,40069,Johnston,Johnston County,06,H1,G4020,None,None,None,A,"POLYGON ((-96.74415 34.17223, -96.74416 34.172..."


In [7]:
geometry_column=county_gdf['geometry']
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x                   y
0    0    -94.470506000000000  40.975042999999999
     1    -94.470500999999999  40.975550999999996
     2    -94.470511999999999  40.978164000000000
     3    -94.470497999999992  40.978803999999997
     4    -94.470501999999996  40.979332999999997
...                       ...                 ...
3232 3894 -84.275637000000003  39.289242000000002
     3895 -84.275549999999996  39.289235999999995
     3896 -84.275515999999996  39.289234000000000
     3897 -84.275508000000002  39.289232999999996
     3898 -84.275143000000000  39.289206999999998

[7781581 rows x 2 columns]

In [8]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.
        And then to find the fraction of time, PM2.5 was above EPA standard
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            pm_above_df: Dataframe consisting the values of the variables interpolated in the county.
    '''   
    
    
    longitude= lat_lon.loc[(var), 'x']  # get the longitude
    extract_val=len(longitude) ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    latitude=lat_lon.loc[(var),'y']    ## get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= temp.interp(longitude=lon_list, latitude=lat_list)
    
## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=year_avg_finer.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)
    
    initial_df=pd.concat(get_vals)
    summary = initial_df.groupby("valid_time").mean(["latitude", "longitude"])
    df=summary

    total_rows=len(df)

    c=0
    for i in range(0, total_rows):
        if df['t'].values[i] > threshold:
            c=c+1

## convert number to a fraction
    frac_time=(c/total_rows)*100
    pm_above_df=pd.DataFrame({'Temp above percentile':[frac_time]})


    return pm_above_df

In [9]:
len(county_gdf)

3233

In [10]:
%%time

## get the values for all counties

df_list_below=[]

for i in range(0,len(county_gdf)): ## loop for the all list of counties
    try:
        df_list_below.append(single_county(i))
    except:
        pass

CPU times: total: 46min 44s
Wall time: 47min 10s


In [11]:
concatenate_df=pd.concat(df_list_below)
final_df=concatenate_df.reset_index(drop=True)
final_df

,Temp above percentile
0,38.592896174863391
1,39.719945355191257
2,46.687158469945359
3,39.241803278688522
4,48.121584699453550
...,...
3228,50.819672131147541
3229,54.439890710382514
3230,41.222677595628419
3231,60.006830601092901


In [12]:
## lets first create a fips column in the original shape file

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

county_gdf = county_gdf.drop([ 'STATEFP','COUNTYFP','COUNTYNS','CNTYIDFP','NAMELSAD','LSAD','CLASSFP','MTFCC',
                                 'CSAFP','CBSAFP','METDIVFP','FUNCSTAT'], axis=1)
county_gdf

,NAME,geometry,fips
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003
...,...,...,...
3228,Eddy,"POLYGON ((-104.09554 32.00000, -104.09586 32.0...",35015
3229,Caddo,"POLYGON ((-98.36984 35.55133, -98.36964 35.551...",40015
3230,Cimarron,"POLYGON ((-102.14252 36.50032, -102.14380 36.5...",40025
3231,Johnston,"POLYGON ((-96.74415 34.17223, -96.74416 34.172...",40069


In [13]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,Temp above percentile
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175,38.592896174863391
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177,39.719945355191257
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097,46.687158469945359
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109,39.241803278688522
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003,48.121584699453550
...,...,...,...,...
3228,Eddy,"POLYGON ((-104.09554 32.00000, -104.09586 32.0...",35015,50.819672131147541
3229,Caddo,"POLYGON ((-98.36984 35.55133, -98.36964 35.551...",40015,54.439890710382514
3230,Cimarron,"POLYGON ((-102.14252 36.50032, -102.14380 36.5...",40025,41.222677595628419
3231,Johnston,"POLYGON ((-96.74415 34.17223, -96.74416 34.172...",40069,60.006830601092901


In [14]:
county_df=merged_df.dropna()
county_df.head()

,NAME,geometry,fips,Temp above percentile
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175,38.592896174863391
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177,39.719945355191257
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097,46.687158469945359
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109,39.241803278688522
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003,48.121584699453550


In [15]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var=county_var.rename(columns={'PM2.5 above threshold':'PM2.5 above percentile'})
county_var

,NAME,fips,Temp above percentile
0,Union,19175,38.592896174863391
1,Van Buren,19177,39.719945355191257
2,Kiowa,20097,46.687158469945359
3,Logan,20109,39.241803278688522
4,Anderson,20003,48.121584699453550
...,...,...,...
3228,Eddy,35015,50.819672131147541
3229,Caddo,40015,54.439890710382514
3230,Cimarron,40025,41.222677595628419
3231,Johnston,40069,60.006830601092901


In [16]:
county_var.to_pickle('temp_above_percentile.pkl')